In [20]:
import pandas as pd
import numpy as np

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

import matplotlib.pyplot as plt
import seaborn as sns

jtplot.style(figsize=(15, 9))

In [10]:
df = pd.read_csv('../data/bt/gld_gdx_aapl_spy__1H.csv', parse_dates=True, index_col=0, header=[0, 1])
df = df['Close']
df.head()

,AAPL,GDX,GLD,SPY
Datetime,,,,
2022-10-07 14:30:00-04:00,139.721893,24.440001,157.720001,361.619995
2022-10-07 15:30:00-04:00,140.080002,24.420000,157.919998,362.799988
2022-10-10 09:30:00-04:00,140.050003,24.250000,155.869995,362.489990
2022-10-10 10:30:00-04:00,140.020004,24.195000,155.419998,360.339996
2022-10-10 11:30:00-04:00,139.747498,24.190001,155.265106,359.050110


In [3]:
df.shape

(3492, 4)

In [11]:
tickers = df.columns

In [12]:
for col in tickers:
    df[f'{col}_return'] = np.log(df[col] / df[col].shift(1))
#     df[f'{col}_direction'] = np.where(df[f'{col}_return'] > 0, 1, 0)

df.dropna(inplace=True)

In [6]:
# df[[ticker+'_direction' for ticker in tickers]].head()

,AAPL_direction,GDX_direction,GLD_direction,SPY_direction
Datetime,,,,
2022-10-07 15:30:00-04:00,1,0,1,1
2022-10-10 09:30:00-04:00,0,0,0,0
2022-10-10 10:30:00-04:00,0,0,0,0
2022-10-10 11:30:00-04:00,0,0,0,0
2022-10-10 12:30:00-04:00,1,0,1,0


In [15]:
df.shape

(3489, 12)

In [13]:
cols = [ticker+'_return' for ticker in tickers]
link = linkage(df[cols], 'ward', 'euclidean')
df['clusters'] = fcluster(link, 3, criterion='maxclust')

In [14]:
df.head()

,AAPL,GDX,GLD,SPY,AAPL_return,GDX_return,GLD_return,SPY_return,clusters
Datetime,,,,,,,,,
2022-10-07 15:30:00-04:00,140.080002,24.420000,157.919998,362.799988,0.002560,-0.000819,0.001267,0.003258,3
2022-10-10 09:30:00-04:00,140.050003,24.250000,155.869995,362.489990,-0.000214,-0.006986,-0.013066,-0.000855,1
2022-10-10 10:30:00-04:00,140.020004,24.195000,155.419998,360.339996,-0.000214,-0.002271,-0.002891,-0.005949,3
2022-10-10 11:30:00-04:00,139.747498,24.190001,155.265106,359.050110,-0.001948,-0.000207,-0.000997,-0.003586,3
2022-10-10 12:30:00-04:00,139.949997,24.155001,155.350006,358.980011,0.001448,-0.001448,0.000547,-0.000195,3


In [15]:
df.groupby('clusters').count()[tickers]

,AAPL,GDX,GLD,SPY
clusters,,,,
1,427,427,427,427
2,90,90,90,90
3,2972,2972,2972,2972
